# Fighting California forest fires using spatial analysis

The state of California has had a dangerous fire season in 2015 and 2016. A standard procedure while fighting these fires is identifying facilities that are at risk and evacuating them. Tasks such as this can be accomplished easily using **spatial analysis** tools available on your GIS. Spatial analysis tools allow overlaying the extent of fire and the locations of the facilities on a map and identifying the ones that fall within the fire's extent.

Thus, this sample demonstrates the application of spatial analysis tools such as **buffer and overlay**.

This notebook describes a scenario wherein an analyst automates the process of identifying facilities at risk from forest fires and sharing this information with other departments such as the fire department, etc.


In [ ]:
import datetime
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [ ]:
password = getpass()
# create a Web GIS object
gis = GIS("https://esriwebgis.webgistesting.net/portal", "atma.mani", password)

## Visualize the fire data
Let us create a map widget to see the fire related information in it's geographic context:

In [ ]:
# find fire items
search_results = gis.content.search('title:*Fires* & owner:atma.mani')
search_results

In [ ]:
fires_facilities_item = search_results[-1]
fires_facilities_item

In [ ]:
for layer in fires_facilities_item.layers:
    print(layer.properties.name)

In [ ]:
infra_flayer = fires_facilities_item.layers[0]
fires_flayer = fires_facilities_item.layers[1]
type(fires_flayer)

In [ ]:
# create a map of our area of interest
map1 = gis.map('Los Angeles', 9)
map1

In [ ]:
# add the active fires layer
map1.add_layer(fires_facilities_item)

## Create a buffer of 4 miles around fire boundaries

Use the **`create_buffers`** function available in the **`arcgis.features.use_proximity`** module.
As an input to the tool, we provide the fires layer as the layer to be buffered.

Feature analysis tools can return (in memory) feature collections as output for immediate consumption, or create a new feature service if the `output_name` parameter is specified.

We specify an output name (with a timestamp) for the service as we may want the buffered fire perimeters to be persisted for bookkeeping purposes, or be shared with others as feature layers or in web maps:

In [ ]:
from arcgis.features.use_proximity import create_buffers
timestamp = '{:%Y_%m_%d_%H_%M_%S}'.format(datetime.datetime.now())
print(timestamp)

In [ ]:
firebuffers = create_buffers(input_layer = fires_flayer, distances = [4], units = 'Miles', 
                             output_name="Fire_Buffers_" + timestamp)

In [ ]:
firebuffers

In [ ]:
# share the layer with public
firebuffers.share(org = True)

In [ ]:
# add risk areas to map
map1.add_layer(firebuffers)

## Perform overlay analysis to extract facilities that fall within the fire buffers

Use **`overlay_layers`** tool under `FeatureAnalysisTools` class just like we did for `create_buffers` tool earlier. The overlay layers tool supports a few overlay types, here we use **`Intersect`** as we need to perform a spatial intersection to identify the facilities that are located within the fire boundaries. To learn more about this operation, refer to the [documentation](https://developers.arcgis.com/rest/analysis/api-reference/overlay-layers.htm).

We specify an output name (with a timestamp) for the service as we want to keep a record of the critical infrastructure within the risk boundaries and share it with others as feature layers or in web maps:

In [ ]:
from arcgis.features  import manage_data

In [ ]:
infrastructure_under_risk = manage_data.overlay_layers(input_layer = firebuffers, 
                                                       overlay_layer = infra_flayer,
                                                       overlay_type="Intersect",
                                                       output_name="At_Risk_Infrastructure_" + timestamp)

In [ ]:
infrastructure_under_risk

In [ ]:
infrastructure_under_risk.share(True)

In [ ]:
infrastructure_under_risk.layers

In [ ]:
map1.add_layer(infrastructure_under_risk)

## Read analysis results as a pandas dataframe for analysis


Let us read this results of the overlay analysis by querying it's layer for attribute data, and display the attribute table to get a sense of the results. Layers can be queried for attributes and geometry using the `query()` method.

The query we ran in the previous step returned a list of dictionaries representing the features. For further analysis or for visualizing the output data as a table, let us convert it into a [pandas](http://pandas.pydata.org/) [dataframe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [ ]:
at_risk_facilities  = infrastructure_under_risk.layers[0]
features = at_risk_facilities.query()

df = features.df # read the returned features as a Pandas dataframe
df

From the data frame, we can see there are 7 features which fell within the fire buffers. To make this result usable, let us export a CSV with only the facility name and other critical details. This CSV can be considered as a valuable information product as a result of this analysis and can be shared with the teams assisting in organizing the firefighting efforts, evacuations, etc.

In [ ]:
# view simplified risk facilities table
df1_simplified = df[['name', 'cat1', 'post_id']]
df1_simplified

In [ ]:
# Export this to a csv file. This CSV can then be shared with fire fighters.
csv_file = r'../../datasets/at_risk_facilities.csv'
df1_simplified.to_csv(csv_file)